In [55]:
import theano
import theano.tensor as T
import theano.gradient


In [107]:
input_size = 4
hidden_size = 3
output_size = 1

In [136]:
parameters = theano.shared(numpy.concatenate(
    [numpy.random.randn(input_size * hidden_size + hidden_size * output_size).astype(theano.config.floatX),
     numpy.zeros(hidden_size + output_size, dtype=theano.config.floatX)])).flatten()

In [208]:
def get_N(parameters):
    index_to = input_size * hidden_size
    Wh = parameters[:index_to].reshape((input_size, hidden_size))

    index_from = index_to
    index_to += hidden_size * output_size
    Wy = parameters[index_from:index_to].reshape((hidden_size, output_size))

    index_from = index_to
    index_to += hidden_size
    bh = parameters[index_from:index_to]

    index_from = index_to
    by = parameters[index_from:]

    x = T.matrix(dtype=theano.config.floatX)
    y = tt.matrix(dtype=theano.config.floatX)

    h = T.nnet.sigmoid(theano.dot(x, Wh) + bh)
    z = theano.dot(h, Wy) + by
    c = (z-y)**2
    
    gs = theano.grad(c, parameters)

    hs = theano.gradient.hessian(c, parameters)

    f = theano.function([x, y], [y, c, gs, hs])

    return f

In [209]:
f = get_network(parameters)

results = f(np.random.randn(batch_size, input_size).astype(theano.config.floatX),
            np.random.randn(batch_size, output_size).astype(theano.config.floatX))
for result_index, result in enumerate(results):
    print(result_index, result.shape)
    


0 (10, 1)
1 ()
2 (19,)
3 (19, 19)


In [210]:
parameters = theano.shared(numpy.concatenate(
    [numpy.random.randn(input_size * hidden_size + hidden_size * output_size).astype(theano.config.floatX),
     numpy.zeros(hidden_size + output_size, dtype=theano.config.floatX)]))
ini_params = parameters

f = get_network(parameters)

batch_size = 10

X = np.random.randn(batch_size, input_size).astype(theano.config.floatX)
Y = np.sum(X,1).reshape(-1,1)
for i in range(10):
    results = f(X, Y)

    inv_hs = np.linalg.pinv(results[-1])
    updates = np.dot(inv_hs, results[-2])
    updates = results[-2]*0.1
    parameters = theano.shared(parameters.get_value()  - updates)
    f = get_network(parameters)
    print(results[1])

3.6305716542225825
2.29765863444209
1.3502252713892087
0.8736594399975921
0.6199375796732984
0.4682681268533166
0.374159146355772
0.3140182523238952
0.27447013684530525
0.2476478981317712


In [224]:
parameters = ini_params

f = get_network(parameters)


#X = np.random.randn(batch_size, input_size).astype(theano.config.floatX)
#Y = np.sum(X,1).reshape(-1,1)
for i in range(10):
    results = f(X, Y)

    inv_hs = np.linalg.inv(results[-1])
    descent_dir =  results[-2] / np.linalg.norm(results[-2])
    updates = np.dot(inv_hs, descent_dir)*0.2
    #updates = results[-2]*0.1
    parameters = theano.shared(parameters.get_value()  - updates)
    f = get_network(parameters)
    print(results[1])

3.6305716542225825
3.4159298061048724
3.2453831992090727
3.0853318821187257
2.933569837563863
2.7890869658982598
2.6508682260094085
2.5155266946454793
2.3814398286502376
2.252543393730584
